In [1]:
import nltk;

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from tokenize import tokenize, untokenize, NUMBER, STRING, NAME, OP
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics


import re, string, random
import gzip
import gensim 
import logging
import pandas as pd

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        #token = re.sub("", "",token)
        #token = re.sub("", "",token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens


if __name__ == "__main__":

    stop_words_ru = stopwords.words('russian')
    stop_words_ru.extend(['что', 'но','это', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на', '...'])

    #Read and Split file
    text_lab4 = open('/content/Lab4_text.txt', 'r').read()
    text_lab4_splitted = text_lab4.split('\n')

    #lab4_cleaned_tokens_list = []
    text_lab4_splitted_tokens = []
    text_lab4_splitted_tokens_cleared = []
    text_lab4_splitted_tokens_cleared_final = []
    
    #Tokenize each line by word
    for tokens in text_lab4_splitted:
      text_lab4_splitted_tokens.append(word_tokenize(tokens))
    
    
    # Remove stop words and punctuation
    for tokens in text_lab4_splitted_tokens:
      for tokens2 in tokens:
        if (tokens2 not in stop_words_ru and tokens2 not in string.punctuation):
          text_lab4_splitted_tokens_cleared.append(tokens2)
      text_lab4_splitted_tokens_cleared_final.append(" ".join(text_lab4_splitted_tokens_cleared))
      text_lab4_splitted_tokens_cleared=[]

    #Prepare 2 sets to merge
    ds_sentence = pd.DataFrame(text_lab4_splitted_tokens_cleared_final, columns=['Sentence']) 
    ds_score = pd.read_csv('/content/scores_train.txt')

    #Concatenate Cleaned Sentences with Scores
    result = pd.concat([ds_sentence, ds_score], axis=1, sort=False)

    #Clean NaN from set    
    result = result.dropna()

    # Create our vectorizer
    vectorizer = CountVectorizer()

    # Get the training vectors
    vectors = vectorizer.fit_transform(result['Sentence'])

    # Build the classifier
    clf = MultinomialNB(alpha=.01)

    #  Train the classifier
    clf.fit(vectors, result['Sentimental'])

    # Get the test vectors
    vectors_test = vectorizer.transform(result['Sentence'])

    # Predict and score the vectors
    pred = clf.predict(vectors_test)
    acc_score = metrics.accuracy_score(result['Sentimental'], pred)
    f1_score = metrics.f1_score(result['Sentimental'], pred, average='macro')

    #Show accuracy:
    print('Total accuracy classification score: {}'.format(acc_score))
    print('Total F1 classification score: {}'.format(f1_score))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Total accuracy classification score: 0.8488
Total F1 classification score: 0.8658512846531389


# New Section